# Assignment - Artificial Intelligence 4

## Assignment Background

As testers you don't have much time, so please feel free to just attempt what you can. A difficult part is gauging the difficulty of an assignment to not bee too easy, too difficult, or too long.

The aim of this assignment is to have you engage in a hands on manner with different sections. Due to the constraint of generating embeddings you will be unable to work with a new dataset.

## Assignment Questions:

## Question 1: 

Things move fast in the machine learning world. The **ESM1b** model was created in *2021*, however it was surplanted in the next year by the **ESM2** model. 

Head to the card of the [**ESM2**](https://huggingface.co/facebook/esm2_t33_650M_UR50D) model on the **Hugging Face** website. You will see if you scroll down that there are multiple versions of the model, with varying layers and parameters, the largest having 15 billion.

We would like you to load the **esm2_t33_650M_UR50D** model into memory, which has comparable parameters to the **ESM1b** model.

In [ ]:
import torch
from transformers import EsmModel, AutoTokenizer

In [ ]:
esm2_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
esm2_model = EsmModel.from_pretrained("facebook/esm2_t33_650M_UR50D")

## Question 2: 

Using the **ESM2** model and tokensier you have just downloaded, create a function that takes a list of protein sequences and a list of amino acid positions as input and returns the embeddings of the amino acids at the specified positions in the sequences. For example, the input ['ACFGPLOIE', 'MOPDEYTWK'] and [1, 3, 5] should return the embeddings of the amino acids at positions 1, 3, and 5 in the sequences.

Please find the sequences in the csv file `assignment_sequences.csv` in the `data` folder.

For the assignment, create a function that generates whole sequences embeddings and finds the amino acids embeddings at position **24** and **102**. Not all the sequences are long enough to have an amino acid at position 102, so you should only return the embeddings for the sequences that have an amino acid at position 102. Store the pair of embeddings in a list, appending it to another list for each sequence.

*Note: The positions are **not** given as zero based indexing*

In [ ]:
seq_df = pd.read_csv("./data/assignment_sequences.csv")
print(seq_df.head(3))

sequences = seq_df['protein_seq'].to_list()

In [ ]:
def get_embeddings_at_positions(sequences, positions, model, tokenizer):
    
    embeddings = []
    
    with torch.no_grad():
        
        for seq in sequences:
            
            tokenized_seq = tokenizer(seq, return_tensors="pt")
            emb = model(**tokenized_seq).last_hidden_state

            nested_emb = []

            for pos in positions:

                if pos < emb.shape[1]:  # Check if the position is within the sequence length
                    emb_at_pos = emb[:, pos, :].cpu().detach()
                    nested_emb.append(emb_at_pos)
            
            embeddings.append(nested_emb)
    
    return embeddings


In [ ]:
positions = [24-1, 102-1]

embeddings = get_embeddings_at_positions(sequences, positions, esm2_model, esm2_tokenizer)

## Question 3

Supervised machine learning is not the only way to explore data. Dimensionality reduction can, such as **principle conponent analysis** or **TSNE**, can also be applied to find clusters in a high dimensional dataset.

**Tasks**
- Load in the test dataset, as seen in the **evaluation** section.
- Reduce the **2560** embedding to **2 dimensions** for plotting
    - Do this using the [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) function from sklearn 
    - And with the [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) function from sklearn
- Turn both of the reductions into a *pandas* dataframe with columns: "Dim1", "Dim2", "gene_ids", "labels" (y)
- Use the dataframes and seaborn to create a scatterplot of the two dimensions, coloured by the labels
- Create a plot for both using the seaborn [joinplot](https://seaborn.pydata.org/generated/seaborn.jointplot.html) with kind "kde"

In [ ]:
import glob
import pickle
import numpy as np

# Load all chunks
chunks = []
for chunk_file in sorted(glob.glob("./data/embeddings_test_chunk_*.pkl")):
    with open(chunk_file, 'rb') as file:
        chunks.append(pickle.load(file))

# Combine all chunks
test_dict = {
    'target_id': [],
    'embeddings': [],
    'labels': []
}

for chunk in chunks:
    test_dict['target_id'].extend(chunk['target_id'])
    test_dict['embeddings'].extend(chunk['embeddings'])
    test_dict['labels'].extend(chunk['labels'])

X = np.array(test_dict['embeddings'])
y = test_dict['labels']
ids = test_dict['target_id']
gene_ids = [re.search(r"(.+(?=:))(:p.)([A-Z][a-z]+)([0-9]+)([A-Z][a-z]+)", i).group(1) for i in ids]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

pca_results = pca.fit_transform(X)

pca_df = pd.DataFrame({"Dim1" : pca_results[:, 0], "Dim2" : pca_results[:, 1], "gene_ids" : gene_ids, "labels" : y})

pca_df.head(3)

In [ ]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, random_state=0)
tnse_results = tsne_model.fit_transform(X) 

tsne_df = pd.DataFrame({"Dim1" : tnse_results[:, 0], "Dim2" : tnse_results[:, 1], "gene_ids" : gene_ids, "labels" : y})

tsne_df.head(3)

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(pca_df, x="Dim1", y="Dim2", hue='labels', alpha=0.5);

In [ ]:
sns.jointplot(pca_df, x="Dim1", y="Dim2", hue='labels', kind='kde', alpha=0.8);

In [ ]:
sns.scatterplot(tsne_df, x="Dim1", y="Dim2", hue='labels', alpha=0.5);

In [ ]:
sns.jointplot(tsne_df, x="Dim1", y="Dim2", hue='labels', kind='kde', alpha=0.8);